Exploring a dataset with information about patients. 
Consider next 8 features to predict the mortality rate:
1. Age
2. CHF KiLLip
3. HBR (b)
4. Systolic AP(b)
5. Creatine in blood
6. EF Percentage
7. White blood cells count
8. Glucose

# 1. Import required libraries and load the dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [ ]:
# Load dataset
raw_data = pd.read_excel('./import/DataSet_V47.xlsx')

1.1. Select features

In [ ]:
columns_in_data = ['Age', 
           'ОССН KiLLip', 
           'ЧСС (b)', 
           'Систолическое АД(b)', 
           'Креатинин', 
           'EF%',
           'Лейкоциты(b)',
           'Глюкоза(a)',
           'начало операции',
           'Дата операции в БД',
           'дата выписки',
           'дата смерти',
           'Исход заболевания']
data=raw_data.loc[(raw_data['STEMI']== 'Да')  & 
                   (raw_data['Наличие в БД']== 'Да')  & 
                  (raw_data['Наличие в файле']== 'Да') &
                   (raw_data['ЧКВ']== 'Да'),
                   columns_in_data]

In [ ]:
display(data.shape)
display(data.describe())

Remove rows with NaN values

In [ ]:
print("Amount of NaN values in different columns before:")
display(data.isna().sum())

columns_to_check = ['Age',
                   'ЧСС (b)', 
                   'Систолическое АД(b)', 
                   'Креатинин',
                   'EF%', 
                   'Лейкоциты(b)',  
                   'Глюкоза(a)']
data_no_nan = data.dropna(subset=columns_to_check)
print("Amount of NaN values in different columns after:")
display(data_no_nan.isna().sum())

Parsing date-time columns into date-time format.

In [ ]:
data_no_nan_copy = data_no_nan.copy()
for col_name in ['Дата операции в БД', 'дата выписки', 'дата смерти']:
    new_col_name = f"{col_name}_parsed"
    data_no_nan_copy[new_col_name] = pd.to_datetime(data_no_nan_copy[col_name], errors='coerce')

Now checking 'дата смерти' and 'Исход заболевания' for inconsistent values.

In [ ]:
query_expression = "(not `дата смерти`.isnull() and `Исход заболевания` != 'умер')"
data_no_nan_copy.query(query_expression)

Drop this row.

In [ ]:
# Applying De Morgan's laws to the query
query_expression_2 = "(`дата смерти`.isnull() or `Исход заболевания` == 'умер')"
# Drop rows that satisfy the query expression
display(data_no_nan_copy.shape)
clean_data = data_no_nan_copy.query(query_expression_2)
display(clean_data.shape)

Keeping only death cases within 55 days from the operation date.

Note: If there is no information about "Дата смерти", but "Исход заболевания" is "умер", we will consider the date from "Дата выписки" as the day of death.

In [ ]:
mask = (
    (clean_data['дата смерти_parsed'].notna()) & 
                (clean_data['дата смерти_parsed'] - clean_data['Дата операции в БД_parsed'] > pd.Timedelta('55 days'))
    | ((clean_data['дата смерти_parsed'].isna()) & 
                (clean_data['Исход заболевания'] == 'умер') & 
                (clean_data['дата выписки_parsed'].notna()) & 
                (clean_data['дата выписки_parsed'] - clean_data['Дата операции в БД_parsed'] > pd.Timedelta('55 days'))
        )
)

rows_to_keep = ~mask

In [ ]:
print(f"Rows to drop: {clean_data[mask].shape[0]}")
ready_data = clean_data[rows_to_keep]
print(f"Rows left in the dataset: {ready_data.shape[0]}")

In [ ]:
renamed_df = ready_data.rename(columns={ 
        'ОССН KiLLip': "CHF KiLLip",
        'ЧСС (b)': "HBR (b)", 
        'Систолическое АД(b)': "Systolic AP(b)", 
        'Креатинин': "Creatinine in blood", 
        'EF%': "EF Percentage", 
        'Лейкоциты(b)': "White blood cells count", 
        'Глюкоза(a)': "Glucose",
        'Исход заболевания': 'Class'
    })
renamed_df['Class'] = renamed_df['Class'].apply(lambda x: 1 if x == 'умер' else 0) 

In [ ]:
features = ["Age", "CHF KiLLip", "HBR (b)", "Systolic AP(b)", "Creatinine in blood", "EF Percentage", "White blood cells count", "Glucose", "Class"]

final_df = renamed_df[features]

In [ ]:
final_df.describe()

In [ ]:
final_df.to_csv('import/dataset_prepared.csv', index=False)

In [ ]:
plt.figure(figsize = (8,4), dpi=100)
ax = plt.axes()
sns.heatmap(final_df.corr(), annot = True, cmap='RdBu', ax=ax)
ax.set_title('Correlation Matrix', weight='bold')
plt.show()

In [ ]:
neg, pos = np.bincount(final_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Detecting and removing outliers

In [ ]:
print(final_df.columns)

In [ ]:
# Dynamically create a figure to hold the subplots based on the number of features
fig, axs = plt.subplots(len(features), 1, figsize=(10, 2*len(features)))  # Adjust the size as needed

# Loop through the features and create a boxplot for each
for i, feature in enumerate(features):
    sns.boxplot(x=final_df[feature], ax=axs[i], orient='h', palette="Set2")
    axs[i].set_title(feature)

plt.tight_layout()
plt.show()

As a part of an experiment we want to consider Killip classes as categorical type.

In [ ]:
final_df['CHF KiLLip'].value_counts()

In [ ]:
pd.crosstab(index=final_df["CHF KiLLip"], columns=final_df["Class"])

### Categorical Data Encoding

In [ ]:
final_df = pd.get_dummies(final_df, drop_first=True, columns=['CHF KiLLip'])
final_df.describe()

In [ ]:
final_df.to_csv('import/dataset_prepared_KiLLip_encoded.csv', index=False)

In [ ]:
plt.figure(figsize = (12,4), dpi=100)
ax = plt.axes()
sns.heatmap(final_df.corr(), annot = True, cmap='RdBu', ax=ax)
ax.set_title('Correlation Matrix', weight='bold')
plt.show()

## Pairplot

Due to the amount of features, we devide the pairplot into four quarters: top left, top right, bottom left, bottom right. And display two of them: top left and bottom left, because bottom right cosists of categorical types (KiLLip).

## Top left

In [ ]:
sns.pairplot(pd.concat([final_df[final_df["Class"] != 1], final_df[final_df["Class"] == 1]]), 
            x_vars = ["Age", "HBR (b)", "Systolic AP(b)", "Creatinine in blood", "EF Percentage", 
                 "White blood cells count"],
            y_vars=["Age", "HBR (b)", "Systolic AP(b)", "Creatinine in blood", "EF Percentage", 
                 "White blood cells count"],
             hue="Class", diag_kind='kde',
             plot_kws = {'marker':'o', 'alpha': 0.8, 's': 80, 'edgecolor': 'k'},
             height = 4,
             corner=True)

### Bottom left

In [ ]:
sns.pairplot(pd.concat([final_df[final_df["Class"] != 1], final_df[final_df["Class"] == 1]]), 
            x_vars = ["Age", "HBR (b)", "Systolic AP(b)", "Creatinine in blood", "EF Percentage", 
                 "White blood cells count"],
            y_vars=["Glucose"],
             hue="Class", diag_kind='kde',
             plot_kws = {'marker':'o', 'alpha': 0.8, 's': 80, 'edgecolor': 'k'},
             height = 4)